In [4]:
import sys
sys.path.append("/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src")
from pathlib import Path
import pandas as pd

from transformers import AutoTokenizer

from data_utils import encode_multilabel, get_folds, encode_df

In [2]:
data_dir = Path("/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/dataset")

data_path = data_dir / "jigsaw22_validation_data.csv"

In [3]:
data = pd.read_csv(data_path)
print(data.shape)
data.head()

(30108, 3)


,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [5]:
def clean(df, col_name):
    df.loc[:, col_name] = df[col_name].str.replace('https?://\S+|www\.\S+', ' social medium ')
    df.loc[:, col_name] = df[col_name].str.replace('\s+', ' ')
    df.loc[:, col_name] = df[col_name].str.strip()
    
    return df

In [7]:
data = clean(data.copy(), col_name='less_toxic')
data = clean(data.copy(), col_name='more_toxic')

data.head()

,worker,less_toxic,more_toxic
0,313,This article sucks woo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness Top of the mornin' my favourit...
2,82,"Western Media? Yup, because every crime in the...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina. Might wa...
4,539,smelly vagina Bluerasberry why don't you be a ...,"hey way to support nazis, you racist"


In [8]:
tokenizer_type = 'roberta-base'  # change if needed

tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)

In [10]:
less_toxic_data = encode_df(data.copy(), tokenizer,
                            col_name='less_toxic', max_length=256, drop=True)
more_toxic_data = encode_df(data.copy(), tokenizer,
                            col_name='more_toxic', max_length=256, drop=True)

100%|██████████| 30108/30108 [00:12<00:00, 2491.30it/s]
/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src/data_utils.py:78: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(col_name, 1, inplace=True)
100%|██████████| 30108/30108 [00:11<00:00, 2551.24it/s]


In [12]:
less_toxic_data.drop('more_toxic', 1, inplace=True)
more_toxic_data.drop('less_toxic', 1, inplace=True)

/Users/avgalichin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/Users/avgalichin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [16]:
less_toxic_data.to_csv(data_dir / 'gen_less_toxic_data.csv', index=False)
more_toxic_data.to_csv(data_dir / 'gen_more_toxic_data.csv', index=False)